In [1]:
import torch
from torch import optim
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import transforms
from torchvision import models
import numpy as np
from tqdm.notebook import tqdm
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
from os import path

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

BATCH_SIZE = 512
NUM_EPOCHS = 100
PRINT_EVERY = NUM_EPOCHS // 100 if NUM_EPOCHS > 100 else 1
TEACHER_PATH = "./teacher.pth"
LR = 0.01
NUM_WORKERS = 1

cuda


In [3]:
def get_acc(net, loader):
    net.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for data in loader:
            images, labels = data
            
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    net.train()
    return 100 * correct / total

In [4]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=True, 
                                          num_workers=NUM_WORKERS)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=BATCH_SIZE, 
                                         shuffle=False, 
                                         num_workers=NUM_WORKERS)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(len(classes))

Files already downloaded and verified
Files already downloaded and verified
10


In [5]:
teacher = models.vgg16_bn(pretrained=True)

# for param in teacher.features.parameters():
#     param.requires_grad = False

teacher.classifier[6] = nn.Linear(4096,10)

# teacher.classifier[6] = nn.Linear(4096,1024)
# teacher.classifier.add_module("head", nn.Linear(1024, 10))


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(teacher.parameters(), lr=LR, momentum=0.9)
# optimizer = optim.Adam(teacher.parameters(), lr=LR)

In [6]:
if not path.exists(TEACHER_PATH):
    t = tqdm(range(NUM_EPOCHS))
    teacher.to(DEVICE)
    for epoch in t:
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            
            optimizer.zero_grad()

            outputs = teacher(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        if (epoch + 1) % PRINT_EVERY == 0:
            acc = get_acc(teacher, testloader)
            print(f'[{epoch + 1}] loss: {running_loss / len(trainloader):0.9f} | accuracy: {acc:0.2f}%')
            running_loss = 0.0

    print("Finished Training")
    torch.save(teacher.state_dict(), TEACHER_PATH)
else:
    print("Loaded saved teacher model")
    teacher.load_state_dict(torch.load(TEACHER_PATH))
    teacher.to(DEVICE)

Loaded saved teacher model


In [ ]:
print(teacher)

In [7]:
TEACHER_NUM_PARAMS = sum(p.numel() for p in teacher.parameters())
print(TEACHER_NUM_PARAMS)

134309962


In [8]:
TEACHER_ACC = get_acc(teacher, testloader)
print(f"Accuracy: {TEACHER_ACC} %")

Accuracy: 90.03 %


In [9]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data    
        outputs = teacher(images.to(DEVICE))    
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions.cpu()):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1
  
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f"Accuracy for class {classname} is: {accuracy}")

Accuracy for class plane is: 92.8
Accuracy for class car is: 94.6
Accuracy for class bird is: 87.3
Accuracy for class cat is: 77.9
Accuracy for class deer is: 89.6
Accuracy for class dog is: 82.9
Accuracy for class frog is: 93.0
Accuracy for class horse is: 92.0
Accuracy for class ship is: 94.7
Accuracy for class truck is: 92.4


In [10]:
class dVGG(nn.Module):
    def __init__(self, a=0, kind=1):
        super().__init__()
        self.one = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            
            nn.MaxPool2d(2, 2),
            
            nn.Flatten(),
            
            nn.Linear(16 * 5 * 5, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

        self.two = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),

            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            
            nn.Flatten(),
            
            nn.Linear(16 * 10 * 10, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

        self.three = nn.Sequential(
            nn.Conv2d(3, 64, 3),
            nn.ReLU(),
            nn.Dropout(),

            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(),
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, 3),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(),
            nn.BatchNorm2d(256),

            nn.Conv2d(256, 512, 3),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(),
            nn.Dropout(),
            nn.BatchNorm2d(512),

            nn.Flatten(),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, 10)
        )
        
        self.criterion_mse = torch.nn.MSELoss()
        self.criterion_ce = torch.nn.CrossEntropyLoss()
        self.a = a
        self.kind = kind

    def forward(self, x):
        if self.kind == 1:
            out = self.one(x)
        elif self.kind == 2:
            out = self.two(x)
        elif self.kind == 3:
            out = self.three(x)
        else:
            raise ValueError("Unexpected `kind`")

        return out
    
    def loss(self, output, teacher_prob, real_label):
        return self.a * self.criterion_ce(output, real_label) + (1 - self.a) * self.criterion_mse(output, teacher_prob)


In [11]:
def train_baseline(kind=3, opt="sgd", coef=1):
    print(f"=== {kind} | {opt} ===")
    criterion = nn.CrossEntropyLoss()
    net = dVGG(1, kind).to(DEVICE)
    
    if opt.lower() == "sgd":
        optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
    if opt.lower() == "adam":
        optimizer = optim.Adam(net.parameters(), lr=LR)

    if not path.exists(f"baseline_{kind}_{opt}.pth"):
        for epoch in tqdm(range(int(NUM_EPOCHS * coef))):
            running_loss = 0.0
            for i, data in enumerate(trainloader):
                inputs, labels = data
                
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            if (epoch + 1) % PRINT_EVERY == 0:
                acc = get_acc(net, testloader)
                print(f'[{epoch + 1}] loss: {running_loss / len(trainloader):0.5f} | accuracy: {acc:0.2f}%')
                running_loss = 0.0

        torch.save(net.state_dict(), f"./baseline_{kind}_{opt}.pth")
        print(f"=== Finished: {kind} | {opt} ===")
    else:
        print("Loaded saved baseline model")
        net.load_state_dict(torch.load(f"baseline_{kind}_{opt}.pth"))
        net.to(DEVICE)

    baseline_acc = get_acc(net, testloader)

    print("Baseline accuracy on test:", baseline_acc, "%")
    return baseline_acc

In [12]:
def distil(a=0, kind=3, opt="sgd", coef=1):
    print(f"=== {a} | {kind} | {opt} ===")
    net = dVGG(a, kind).to(DEVICE)
    if opt.lower() == "sgd":
        optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
    if opt.lower() == "adam":
        optimizer = optim.Adam(net.parameters(), lr=LR)

    for epoch in tqdm(range(int(NUM_EPOCHS * coef))):
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()

            outputs_teacher = teacher(inputs)
            outputs = net(inputs)

            loss = net.loss(outputs, outputs_teacher, labels)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        if (epoch + 1) % PRINT_EVERY == 0:
            acc = get_acc(net, testloader)
            print(f'[{epoch + 1}] loss: {running_loss / len(trainloader):0.5f} | accuracy: {acc:0.2f}%')
            running_loss = 0.0

    learner_acc = get_acc(net, testloader)
    torch.save(net.state_dict(), f"./distilled_{a}_{kind}_{opt}.pth")

    learner_num_params = sum(p.numel() for p in net.parameters())
    print(f"=== Finished: {a} | {kind} | {opt} ===")
    print("\tTotal number of teacher params:", TEACHER_NUM_PARAMS)
    print("\tTotal number of learner params:", learner_num_params)
    print("\tTotal reduction:", (TEACHER_NUM_PARAMS - learner_num_params) / TEACHER_NUM_PARAMS * 100, "%")
    print("\tTeacher  accuracy on test:", TEACHER_ACC, "%")
    print("\tLearner  accuracy on test:", learner_acc, "%")
    print("\tBaseline accuracy on test:", BASELINE_ACC, "%")
    print("\tDiff:", TEACHER_ACC - learner_acc, learner_acc - BASELINE_ACC)
    print()

In [13]:
BASELINE_ACC = train_baseline(coef=1)

=== 3 | sgd ===


  0%|          | 0/100 [00:00<?, ?it/s]

[1] loss: 2.11921 | accuracy: 12.37%
[2] loss: 1.69053 | accuracy: 21.20%
[3] loss: 1.47043 | accuracy: 33.19%
[4] loss: 1.33389 | accuracy: 43.50%
[5] loss: 1.22507 | accuracy: 50.28%
[6] loss: 1.14128 | accuracy: 50.79%
[7] loss: 1.07695 | accuracy: 55.64%
[8] loss: 1.01381 | accuracy: 61.05%
[9] loss: 0.96132 | accuracy: 62.05%
[10] loss: 0.91014 | accuracy: 66.29%
[11] loss: 0.86752 | accuracy: 68.11%
[12] loss: 0.83093 | accuracy: 70.35%
[13] loss: 0.79132 | accuracy: 71.68%
[14] loss: 0.75736 | accuracy: 71.47%
[15] loss: 0.73473 | accuracy: 71.54%
[16] loss: 0.70967 | accuracy: 73.56%
[17] loss: 0.68172 | accuracy: 74.61%
[18] loss: 0.65627 | accuracy: 76.05%
[19] loss: 0.63578 | accuracy: 76.59%
[20] loss: 0.62276 | accuracy: 75.67%
[21] loss: 0.59852 | accuracy: 76.78%
[22] loss: 0.58718 | accuracy: 76.85%
[23] loss: 0.56098 | accuracy: 78.85%
[24] loss: 0.54496 | accuracy: 77.79%
[25] loss: 0.52900 | accuracy: 79.15%
[26] loss: 0.50955 | accuracy: 79.96%
[27] loss: 0.49567 | 

In [ ]:
for a in (0, 0.1, 0.5, 0.7, 0.9):
    for opt in ("sgd", "adam"):
        distil(a=a, kind=3, opt=opt, coef=1)